In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 15 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.2
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_G_nybnsb
H2O cluster total nodes:,1
H2O cluster free memory:,915 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


##### Use Q1 as training data and Q2 as testing data.

In [2]:
train = h2o.import_file("train_h2o.csv")
test = h2o.import_file("test_h2o.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
y = "Interest_Rate"

In [4]:
aml = H2OAutoML(seed = 1, project_name = "assignment_3_h2o")
aml.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [5]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20181129_024605,0.0906221,0.301035,0.0906221,0.230991,0.0443374
StackedEnsemble_AllModels_AutoML_20181129_024605,0.0906221,0.301035,0.0906221,0.230991,0.0443374
XRT_1_AutoML_20181129_024605,0.0924068,0.303985,0.0924068,0.233159,0.0447719
DRF_1_AutoML_20181129_024605,0.0939478,0.306509,0.0939478,0.235459,0.0451614
DRF_1_AutoML_20181129_042418,0.0940282,0.30664,0.0940282,0.235614,0.0451782
DRF_1_AutoML_20181129_042500,0.0940766,0.306719,0.0940766,0.235646,0.0451898
GLM_grid_1_AutoML_20181129_024605_model_1,0.0950989,0.308381,0.0950989,0.235229,0.0452053


In [6]:
pred = aml.predict(test)
pred.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
5.7886
5.77839
5.79354
5.77005
5.91066
5.88596
5.84118
5.83156
5.88919
5.90307


In [7]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.09062206722153747
RMSE: 0.30103499335050315
MAE: 0.23099075760032312
RMSLE: 0.044337414502309484
R^2: 0.2627142578998969
Mean Residual Deviance: 0.09062206722153747
Null degrees of freedom: 405678
Residual degrees of freedom: 405675
Null deviance: 56732.94213759017
Residual deviance: 36763.4696083661
AIC: 177218.51351249518


##### Use cross-validated metrics.

In [8]:
df =  h2o.import_file("train_h2o.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
splits = df.split_frame(ratios = [0.8], seed = 1)
training = splits[0]
testing = splits[1]

In [10]:
aml2 = H2OAutoML(seed = 1, project_name = "Q1_cross_validated")
aml2.train(y = y, training_frame = df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
aml2.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20181129_035203,0.0687883,0.262275,0.0687883,0.193903,0.0391119
StackedEnsemble_AllModels_AutoML_20181129_035203,0.0687883,0.262275,0.0687883,0.193903,0.0391119
DRF_1_AutoML_20181129_035203,0.0692005,0.26306,0.0692005,0.194522,0.0392329
DRF_1_AutoML_20181129_052541,0.0692014,0.263062,0.0692014,0.194519,0.0392334
XRT_1_AutoML_20181129_052541,0.0694419,0.263518,0.0694419,0.194954,0.0393203
XRT_1_AutoML_20181129_035203,0.0695808,0.263782,0.0695808,0.195153,0.0393598
GLM_grid_1_AutoML_20181129_035203_model_1,0.0785475,0.280263,0.0785475,0.207376,0.0417873
GLM_grid_1_AutoML_20181129_052541_model_1,0.0785475,0.280263,0.0785475,0.207376,0.0417873


In [12]:
perf = aml2.leader.model_performance(testing)
perf


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 0.04969705340454395
RMSE: 0.22292835935462305
MAE: 0.16750499838322996
RMSLE: 0.03335339367770484
R^2: 0.629109086591956
Mean Residual Deviance: 0.04969705340454395
Null degrees of freedom: 70133
Residual degrees of freedom: 70130
Null deviance: 9397.803624026541
Residual deviance: 3485.4531434742858
AIC: -11487.246784618035
